In [1]:
!pip install sqlite3


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [6]:
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
from PIL import Image, ImageTk
import sqlite3

# Hàm tạo bảng và kết nối đến cơ sở dữ liệu
def create_database():
    conn = sqlite3.connect('my_database.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS DiemTichLuy (
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            TenKhachHang TEXT,
            SoDienThoai TEXT,
            Diem INTEGER
        )
    ''')
    conn.commit()
    conn.close()

# Hàm thêm điểm tích lũy
def add_score():
    ten_khach_hang = ten_khach_hang_entry.get("1.0", "end-1c")
    so_dien_thoai = so_dien_thoai_entry.get("1.0", "end-1c")

    if not so_dien_thoai or so_dien_thoai == '0' or len(so_dien_thoai) > 11:
        messagebox.showerror("Lỗi", "Số điện thoại không hợp lệ.")
        return

    conn = sqlite3.connect('my_database.db')
    cursor = conn.cursor()
    select_query = "SELECT ID, TenKhachHang, SoDienThoai, Diem FROM DiemTichLuy WHERE SoDienThoai = ?"
    cursor.execute(select_query, (so_dien_thoai,))
    existing_row = cursor.fetchone()

    if not existing_row:
        insert_query = "INSERT INTO DiemTichLuy (TenKhachHang, SoDienThoai, Diem) VALUES (?, ?, ?);"
        cursor.execute(insert_query, (ten_khach_hang, so_dien_thoai, 1))
        conn.commit()

        message_label.config(text="Đã thêm điểm tích lũy.")
        message_label.place(relx=0.5, rely=0.23, anchor=tk.CENTER)
        root.after(500, lambda: message_label.config(text=""))
    else:
        updated_score = existing_row[3] + 1

        if updated_score == 10:
            rewarded_customer = f"Khách hàng {existing_row[1]}"
            messagebox.showinfo("Thông báo", f"{rewarded_customer} đã nhận được một phần quà")
            reset_scores(so_dien_thoai)
        else:
            update_query = "UPDATE DiemTichLuy SET Diem = ? WHERE ID = ?;"
            cursor.execute(update_query, (updated_score, existing_row[0]))
            conn.commit()

        message_label.config(text="Đã thêm điểm tích lũy.")
        message_label.place(relx=0.5, rely=0.23, anchor=tk.CENTER)
        root.after(850, lambda: message_label.config(text=""))

    conn.close()
    view_scores()
    highlight_row(so_dien_thoai)

    ten_khach_hang_entry.delete("1.0", tk.END)
    so_dien_thoai_entry.delete("1.0", tk.END)

# Hàm xem điểm tích lũy
def view_scores():
    conn = sqlite3.connect('my_database.db')
    cursor = conn.cursor()
    select_query = "SELECT ROW_NUMBER() OVER(ORDER BY ID) as STT, TenKhachHang, SoDienThoai, Diem FROM DiemTichLuy;"
    cursor.execute(select_query)
    rows = cursor.fetchall()
    conn.close()

    for i in tree.get_children():
        tree.delete(i)

    if rows:
        for row in rows:
            data = (row[0], row[1], row[2], str(row[3]))
            tree.insert("", "end", values=data)
    else:
        messagebox.showinfo("Thông báo", "Không có dữ liệu.")

# Hàm xóa dữ liệu
def delete_data():
    choice = simpledialog.askstring("Xóa dữ liệu", "Chọn 1 để xóa tất cả hoặc nhập số điện thoại để xóa theo số điện thoại:")
    
    if choice == '1':
        conn = sqlite3.connect('my_database.db')
        cursor = conn.cursor()
        delete_query = "DELETE FROM DiemTichLuy"
        cursor.execute(delete_query)
        conn.commit()
        conn.close()
        view_scores()
    elif choice:
        conn = sqlite3.connect('my_database.db')
        cursor = conn.cursor()
        delete_query = "DELETE FROM DiemTichLuy WHERE SoDienThoai = ?"
        cursor.execute(delete_query, (choice,))
        conn.commit()
        conn.close()
        view_scores()

# Hàm reset điểm tích lũy của một người
def reset_scores(phone_number):
    conn = sqlite3.connect('my_database.db')
    cursor = conn.cursor()
    update_query = "UPDATE DiemTichLuy SET Diem = 0 WHERE SoDienThoai = ?;"
    cursor.execute(update_query, (phone_number,))
    conn.commit()
    conn.close()
    view_scores()

# Hàm kết nối đến cơ sở dữ liệu
def connect_to_database():
    try:
        conn = sqlite3.connect('my_database.db')
        cursor = conn.cursor()
        return conn, cursor
    except Exception as e:
        messagebox.showerror("Lỗi", f"Lỗi kết nối đến cơ sở dữ liệu: {str(e)}")
        return None, None

# Hàm bôi xanh dòng tương ứng với số điện thoại
def highlight_row(phone_number):
    for item in tree.get_children():
        if tree.item(item, "values")[2] == phone_number:
            tree.selection_set(item)
            tree.see(item)

# Tạo cơ sở dữ liệu
create_database()

# Tạo giao diện
root = tk.Tk()
root.title("Quản lý điểm tích lũy")

# Cập nhật hình nền
background_image_path = "C:\\Users\\DELL\\Downloads\\f966c4115bc38f9dd6d2.jpg"  # Đặt đường dẫn đến hình nền của bạn ở đây
background_image = Image.open(background_image_path)

screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

background_label = tk.Label(root)
background_label.place(relwidth=1, relheight=1)

background_image = background_image.resize((screen_width, screen_height))
background_photo = ImageTk.PhotoImage(background_image)
background_label.config(image=background_photo)
background_label.image = background_photo

frame = tk.Frame(root)
frame.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

tk.Label(frame, text="Tên khách hàng:", font=("Arial", 15)).grid(row=0, column=0, pady=(10, 10))
ten_khach_hang_entry = tk.Text(frame, height=1, width=50, font=("Arial", 11))
ten_khach_hang_entry.grid(row=0, column=1, pady=10)

tk.Label(frame, text="Số điện thoại:", font=("Arial", 15)).grid(row=1, column=0, pady=(10, 10))
so_dien_thoai_entry = tk.Text(frame, height=1, width=50, font=("Arial", 11))
so_dien_thoai_entry.grid(row=1, column=1, pady=10)

add_button = tk.Button(frame, text="Thêm điểm tích lũy", font=("Arial", 13), command=add_score)
add_button.grid(row=2, column=0, columnspan=3, pady=30)

view_button = tk.Button(frame, text="Xem điểm tích lũy", font=("Arial", 13), command=view_scores)
view_button.grid(row=3, column=0, columnspan=3)

delete_button = tk.Button(frame, text="Xóa dữ liệu", font=("Arial", 13), command=delete_data)
delete_button.grid(row=4, column=0, columnspan=3)

tree_frame = tk.Frame(frame)
tree_frame.grid(row=5, column=0, columnspan=2, pady=(20, 0))

# Tạo thanh trượt
tree_scroll = tk.Scrollbar(tree_frame, orient="vertical")
tree_scroll.pack(side="right", fill="y")

# Tạo Treeview
style = ttk.Style()
style.configure("Treeview.Heading", font=("Arial", 13))
style.configure("Treeview", font=("Arial", 11))
tree = ttk.Treeview(tree_frame, columns=("STT", "Tên khách hàng", "Số điện thoại", "Điểm tích lũy"), yscrollcommand=tree_scroll.set)
tree.heading("#0", text="")
tree.heading("#1", text="STT")
tree.heading("#2", text="Tên khách hàng")
tree.heading("#3", text="Số điện thoại", anchor=tk.CENTER)
tree.heading("#4", text="Điểm tích lũy")
tree.column("#0", width=0)
tree.column("#1", width=80)
tree.column("#2", width=250)
tree.column("#3", width=250, anchor=tk.CENTER)
tree.column("#4", width=150)
tree.pack(fill="both", expand=1)

# Cấu hình thanh trượt
tree_scroll.config(command=tree.yview)
for col in ("#1", "#2", "#3", "#4"):
    tree.heading(col, anchor=tk.CENTER)
    tree.column(col, anchor=tk.CENTER)

# Label để hiển thị thông báo
message_label = tk.Label(frame, text="", font=("Arial", 15))
message_label.grid(row=6, column=0, columnspan=2, pady=10)

root.mainloop()
